In [12]:
import numpy as np
from math import exp, sqrt

In [13]:
""" 
Function to calculate option price using the CRR binomial tree model.

Inputs:
S: Initial stock price
K: Strike price
T: Time to maturity (years)
r: Risk-free rate
v: Volatility
N: Number of steps in the binomial tree
option_type1: 'european' or 'american'
option_type2: 'call' or 'put'

"""

def binomial_tree_crr(S, K, T, r, v, N, option_type1="european", option_type2="call"):

    #Calculating dt (small time intervals)
    dt = T / N 
    
    #Calculating probabilities
    u = exp(v * sqrt(dt))
    d = 1 / u
    p = (exp(r * dt) - d) / (u - d)

    #Dictionary to store values of each node
    val = {}

    #Function to calculate the price (recurrsion)
    def compute_price(i, j):
        
        if (i, j) in val:
            return val[(i, j)]

        #Terminal node
        if i == N:
            stock_price = S * (u ** j) * (d ** (N - j))
            if option_type2 == "call":
                option_value = max(stock_price - K, 0)
            else:
                option_value = max(K - stock_price, 0)
        else:
            up_value = compute_price(i + 1, j + 1)
            down_value = compute_price(i + 1, j)
            discounted_value = exp(-r * dt) * (p * up_value + (1 - p) * down_value) #dicounting back
            stock_price = S * (u ** j) * (d ** (i - j))
            
            if option_type1 == "american":
                if option_type2 == "call":
                    intrinsic_value = max(stock_price - K, 0)
                else:
                    intrinsic_value = max(K - stock_price, 0)
                
                option_value = max(discounted_value, intrinsic_value)
            else:
                option_value = discounted_value
                
        val[(i, j)] = option_value
        return option_value
    
    option_price = compute_price(0, 0)
    
    #Greeks calculations (delta, gamma, theta)
    delta = (compute_price(1, 1) - compute_price(1, 0)) / (S * (u - d))
    gamma = ((compute_price(2, 2) - compute_price(2, 1)) / (S * u - S) - (compute_price(2, 1) - compute_price(2, 0)) / (S - S * d)) / ((S * u - S * d) / 2)
    theta = (compute_price(2, 0) - option_price) / (2 * dt)
    
    return option_price, delta, gamma, theta

In [14]:
#Function to compute rho & vega
def compute_vega_rho(S, K, T, r, v, N, option_type1, option_type2):
    dt_v = 0.01
    dt_r = 0.01
    
    vega = (binomial_tree_crr(S, K, T, r, v + dt_v, N, option_type1, option_type2)[0] - binomial_tree_crr(S, K, T, r, v - dt_v, N, option_type1, option_type2)[0]) / (2*dt_v)
    
    rho = (binomial_tree_crr(S, K, T, r + dt_r, v, N, option_type1, option_type2)[0] - binomial_tree_crr(S, K, T, r - dt_r, v, N, option_type1, option_type2)[0]) / (2*dt_r)
    
    return vega, rho

In [15]:
#Example 
S = 100  
K = 100 
T = 1   
r = 0.05 
v = 0.2  
N = 3   

#Pricing European Call and Put options
european_call = binomial_tree_crr(S, K, T, r, v, N, "european", "call")
european_put = binomial_tree_crr(S, K, T, r, v, N, "european", "put")
vega_european_call, rho_european_call = compute_vega_rho(S, K, T, r, v, N, "european", "call")
vega_european_put, rho_european_put = compute_vega_rho(S, K, T, r, v, N, "european", "put")

#Pricing American Call and Put options
american_call = binomial_tree_crr(S, K, T, r, v, N, "american", "call")
american_put = binomial_tree_crr(S, K, T, r, v, N, "american", "put")
vega_american_call, rho_american_call = compute_vega_rho(S, K, T, r, v, N, "american", "call")
vega_american_put, rho_american_put = compute_vega_rho(S, K, T, r, v, N, "american", "put")

print(f"European Call: {european_call}, Vega: {vega_european_call}, Rho: {rho_european_call}")
print(f"European Put: {european_put}, Vega: {vega_european_put}, Rho: {rho_european_put}")
print(f"American Call: {american_call}, Vega: {vega_american_call}, Rho: {rho_american_call}")
print(f"American Put: {american_put}, Vega: {vega_american_put}, Rho: {rho_american_put}")

European Call: (11.043871091951113, 0.6140855606115924, 0.09698709885828748, -16.56580663792667), Vega: 41.077650303367626, Rho: 50.36202762258899
European Put: (6.166813542022532, -0.3859144393884075, 0.0769870988582875, 19.2024468061685), Vega: 41.07765030336714, Rho: -44.76250021778378
American Call: (11.043871091951113, 0.6140855606115924, 0.09698709885828748, -16.56580663792667), Vega: 41.077650303367626, Rho: 50.36202762258899
American Put: (6.499559886616256, -0.4179557255073039, 0.09008386293564491, 21.18260921603528), Vega: 41.82047487432281, Rho: -40.511081081587854
